In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField


In [2]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("feature terminal")
         .getOrCreate())

25/02/13 17:20:32 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
25/02/13 17:20:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8f41a274-58b8-47ac-840a-1e4e20e9a3fe;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [3]:
def execute_query(query: str):
    spark.sql(query).show(truncate=False)

In [4]:
df = spark.read \
    .format("iceberg") \
    .load("s3a://commerce/warehouse/payment/transactions/")
df.createOrReplaceTempView('transactions')

25/02/13 17:20:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
execute_query("""select * from transactions""")

+-----+-------------------+-----------+-----------+---------+--------------------------+--------------------------+
|tx_id|tx_datetime        |customer_id|terminal_id|tx_amount|row_created_timestamp     |row_updated_timestamp     |
+-----+-------------------+-----------+-----------+---------+--------------------------+--------------------------+
|1    |2025-01-28 14:42:45|7          |4          |897.34   |2025-02-13 09:26:11.480844|2025-02-13 09:26:11.480844|
|6    |2025-02-03 00:53:15|8          |2          |245.67   |2025-02-13 09:26:11.480844|2025-02-13 09:26:11.480844|
|3    |2025-01-19 00:02:10|0          |3          |67.89    |2025-02-13 09:26:11.480844|2025-02-13 09:26:11.480844|
|5    |2025-01-27 16:15:00|9          |1          |876.21   |2025-02-13 09:26:11.480844|2025-02-13 09:26:11.480844|
|9    |2025-01-25 19:45:00|5          |0          |639.50   |2025-02-13 09:26:11.480844|2025-02-13 09:26:11.480844|
|4    |2025-01-29 10:53:40|4          |0          |299.99   |2025-02-13 

In [ ]:
execute_query("""
CREATE TABLE IF NOT EXISTS feature.feature_terminal (
    terminal_id INT,
    terminal_id_nb_tx_1day_window INT,
    terminal_id_risk_1day_window FLOAT, 
    terminal_id_nb_tx_7day_window INT,
    terminal_id_risk_7day_window FLOAT,
    terminal_id_nb_tx_30day_window INT,
    terminal_id_risk_30day_window FLOAT,
    x_location FLOAT,
    y_location FLOAT,
    row_created_timestamp TIMESTAMP,
    row_updated_timestamp TIMESTAMP
)
USING iceberg
""").show(truncate=False)
